In [ ]:
"""
Retriever tools for multi-namespace RAG
"""
from langchain_core.tools import Tool
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from config import aws_region, bedrock_model, embedding_model, temperature, max_tokens, pinecone_api_key, pinecone_index_name, namespaces
from utils import get_bedrock_embeddings




In [ ]:
def create_retriever_func(namespace_key: str):
        def retriever_func(query: str) -> str:
            print(f"📚 Using {namespace_key} RAG Retriever tool")
            retriever = vector_stores[namespace_key].as_retriever()
            docs = retriever.invoke(query)
            return "\n".join([doc.page_content for doc in docs])
        return retriever_func

In [ ]:
def create_pinecone_retriever_tools(config: Config) -> list:
    
    """
    Create retriever tools for each namespace
    Args:
        config: Application configuration
    Returns:
        List of retriever tools
    """
    
    # Initialize Pinecone
    pc = Pinecone(api_key=pinecone_api_key)
    index = pc.Index(config.pinecone_index_name)
    
    # Get embeddings
    embeddings = get_bedrock_embeddings(config)
    
    # Create vector stores for each namespace
    vector_stores = {}
    for namespace_key, namespace_value in config.namespaces.items():
        vector_stores[namespace_key] = PineconeVectorStore(
            index=index,
            embedding=embeddings,
            namespace=namespace_value,
            text_key="text"
        )
    
    # Create retriever functions
    def create_retriever_func(namespace_key: str):
        def retriever_func(query: str) -> str:
            print(f"📚 Using {namespace_key} RAG Retriever tool")
            retriever = vector_stores[namespace_key].as_retriever()
            docs = retriever.invoke(query)
            return "\n".join([doc.page_content for doc in docs])
        return retriever_func
    
    # Tool descriptions
    tool_descriptions = {
        "US": """
This tool provides authoritative, country-specific guidance based on a comprehensive analysis of U.S. pharmaceutical marketing, regulatory frameworks, and compliance requirements. It summarizes how prescription-drug promotion is governed in the United States, covering the strict oversight applied by the Food & Drug Administration (FDA), Federal Trade Commission (FTC), and federal fraud-and-abuse laws.
""",
        "india": """
This tool provides authoritative, country-specific compliance guidance based on the AdvaMed Code of Ethics on Interactions with Health Care Professionals in India. It outlines India's ethical, regulatory, and industry-governed standards for how medical-technology and pharmaceutical companies must interact with healthcare professionals and institutions.
""",
        "russia": """
This tool provides authoritative, country-specific compliance guidance based on the Handbook of Pharma and MedTech Compliance – Russia. It summarizes Russia's legal and regulatory framework governing the promotion of medicines and medical devices, including advertising rules, restrictions on HCP interactions, gifts, samples, sponsorships, clinical-trial engagements, anti-bribery provisions, and administrative/criminal liabilities.
""",
        "canada": """
This tool provides authoritative, country-specific guidance based on the Canadian PBC Society Policy on Engaging with the Pharmaceutical Industry. It summarizes Canada-focused ethical, transparency, and governance principles that guide how patient organizations and non-profits may interact with pharmaceutical manufacturers, distributors, and related healthcare companies.
""",
        "japan": """
This tool provides authoritative, country-specific compliance guidance based on the Japan Pharmaceutical Manufacturers Association (JPMA) Code of Practice and Promotion Code for Prescription Drugs. It summarizes Japan's ethical, regulatory, and industry self-governance standards that govern interactions between pharmaceutical companies and healthcare professionals, researchers, medical institutions, wholesalers, and patient organizations.
"""
    }
    
    # Create tools
    tools = []
    for namespace_key in config.namespaces.keys():
        tool = Tool(
            name=f"{namespace_key}_RAGRetriever",
            description=tool_descriptions[namespace_key],
            func=create_retriever_func(namespace_key)
        )
        tools.append(tool)
    
    return tools